In [1]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

rank = 0 
sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from mymodule import PipeLine, grid_serch_cv

## グリッドサーチによるハイパーパラメータ調整
グリッドサーチ「なし」と「あり」で効果を確認

In [ ]:
# グリッドサーチの関数の中身
def grid_serch_cv(pack, param_grid, model, score='accuracy'):
    gs_model = GridSearchCV(estimator=model(), 
                    param_grid=param_grid,  # 設定した候補を代入
                    scoring=score, # デフォルトではaccuracyを基準に探してくれる
                    refit=True,
                    cv=10,
                    n_jobs=-1)
    # 訓練データで最適なパラメータを交差検証する
    gs_model.fit(pack[0], pack[2])  # x_train = pack[0], y_train = pack[2]
    evaluations(gs_model, *pack)
    return gs_model

グリッドサーチ<span style = "color: orange;">なし</span>の場合

In [5]:
df = pd.read_csv('../data/train.csv')

# パイプライン
pipe = PipeLine()
pipe(df)
model = pipe.training(valid='fold_out_split', Model=SVC)

,accuracy_score,precision_score,recall_score,f1_score
train,0.750557,0.774892,0.748954,0.761702
test,0.704663,0.773913,0.741667,0.757447


グリッドサーチ<span style = "color: orange;">あり</span>の場合

In [3]:
# sklearnの使用するモデル
model = SVC  # クラスの()いらない

#################### グリッドサーチの探索候補を用意 #################
param_range = [0.001, 0.01, 0.1, 1.0, 10]
param_grid = [{'C': param_range,  # 正則化パラメータの係数を探索
               'kernel': ['linear']},  # Linearカーネル
              {'C': param_range, 
               'gamma': param_range,  # カーネル係数を探索
               'kernel': ['rbf']}]  # rbfカーネル

############################ パイプライン ############################
pipe = PipeLine()
pipe(df)
pack = pipe.fold_out_split()
# グリッドサーチの出番
gs_model = grid_serch_cv(pack, param_grid, model)

,accuracy_score,precision_score,recall_score,f1_score
train,0.817372,0.861751,0.782427,0.820175
test,0.813472,0.850000,0.850000,0.850000


精度が向上してる<br>
※ もともと精度が高いデータセットの場合<span style = "color: orange;">精度が下がる場合がある</span><br>
（パラメータ検証のためデータセットを削っているためと思われ)
>
自作した関数では最適なモデルとパラメータを出力するようになってる


In [4]:
print('ベストなパラメータ: ',gs_model.best_params_)
gs_model

ベストなパラメータ:  {'C': 0.1, 'kernel': 'linear'}


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [0.001, 0.01, 0.1, 1.0, 10],
                          'kernel': ['linear']},
                         {'C': [0.001, 0.01, 0.1, 1.0, 10],
                          'gamma': [0.001, 0.01, 0.1, 1.0, 10],
                          'kernel': ['rbf']}],
             scoring='accuracy')